# Creacion de Csv de Nodos para el grafo de recurrencia.

## Importanciones

In [1]:
import pandas as pd

## Carga del Dataset 

En esta parte cargamos los datasets y añadimos las columnas extras necesarias:
  - conexiones --> dataset_final.csv
  - sitios --> ma_sitios.csv  --> Añadir columna de ubicacion basada en su latitud y longitud

###    Conexiones(Carga y Procesamiento)

In [2]:
conexiones = pd.DataFrame.from_csv("dataset_final.csv", sep = ",")
conexiones.head(3)

,datetime,codigo,pauta_id,sitio_id,mac_usr,ip_local,ip_usr,so,navegador,url_ref,tiempo_espera,fabricante-dispositivo
0,2015-05-15 20:16:35,1,11,2,94:01:C2:97:96:CB,10.5.8.1,10.5.10.95,Android,SAFARI,http://10.5.8.1/status,00:00:00,"Samsung Electronics Co.,Ltd"
1,2015-05-15 20:18:15,2,5,2,8C:BF:A6:17:60:A0,10.5.8.1,10.5.9.24,Android,CHROME,http://10.5.8.1/status,00:00:00,"Samsung Electronics Co.,Ltd"
2,2015-05-15 20:18:24,3,19,3,74:AA:FE:A5:7E:90,10.5.0.1,10.5.0.176,Android,CHROME,http://10.5.0.1/status,00:00:00,NaN


In [3]:
#conexiones["datetime"] = pd.to_datetime(conexiones["datetime"])
conexiones2 = conexiones[["datetime", "mac_usr", "sitio_id"]]
conexiones2.head(3)

,datetime,mac_usr,sitio_id
0,2015-05-15 20:16:35,94:01:C2:97:96:CB,2
1,2015-05-15 20:18:15,8C:BF:A6:17:60:A0,2
2,2015-05-15 20:18:24,74:AA:FE:A5:7E:90,3


###  Truncar hora al valor de 15 mins más bajo

In [4]:
def truncarFecha(fecha):
    dia,time = fecha.split(" ")
    hora,minuto,segundo = time.split(":")
    aux = {0: "00", 1: "15", 2: "30", 3:"45"}
    minuto2 = aux[int(minuto)//15]
    return dia + " " + hora +":"+ minuto2 + ":00"


In [5]:
conexiones2["datetime"] = pd.to_datetime(conexiones2["datetime"].apply(lambda fecha: truncarFecha(fecha)))
conexiones2.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,datetime,mac_usr,sitio_id
0,2015-05-15 20:15:00,94:01:C2:97:96:CB,2
1,2015-05-15 20:15:00,8C:BF:A6:17:60:A0,2
2,2015-05-15 20:15:00,74:AA:FE:A5:7E:90,3
3,2015-05-15 20:15:00,34:BE:00:05:16:18,2
4,2015-05-15 20:15:00,BC:6E:64:71:73:97,2


## Carga y procesamiento de sitios

In [6]:
sitios = pd.DataFrame.from_csv("ma_sitios2.csv", sep = ",").reset_index()
sitios.head(3)

,sitio_id,nombre,siglas,ruc,ciudad,nombre_ubicacion,contacto,direccion,email,telefono,...,transparencia,longitud,latitud,ip,puerto,fecha,usuario_modif,bienvenida,formulario,router
0,1,ALBAN BORJA,ALBAN BORJ,920124541001,Guayaquil,Guayaquil,Fabricio Matamoros,Av. Carlos Julio Arosemena,administracion@albanborja.com,923154211,...,0,-2.168677,-79.916769,186.3.137.95,NaN,2016-09-19 13:28:32,fmatamoros,0,0,Mikrotik
1,2,VILLAGE PLAZA,VILLA,0,Guayaquil,Guayaquil,Gabriela,SAMBORONDON,NaN,NaN,...,0,-2.140406,-79.866376,186.3.144.18,NaN,2016-07-20 17:23:50,fmatamoros,0,0,NaN
2,3,CITY MALL,CITY M.,0,Guayaquil,Guayaquil,Gabriela,Alborada,NaN,NaN,...,0,-2.140951,-79.911774,186.3.153.140,NaN,2015-12-22 10:21:54,fmatamoros,0,0,NaN


In [7]:
places =[["Urdesa", [[-79.915581,-2.178119], [-79.901204,-2.146938]]],
             ["MallSur", [[-79.902062,-2.232042], [-79.894681,-2.226038]]],
             ["CityMall", [[-79.912663,-2.143936], [-79.905968,-2.137932]]],
             ["AlbanBorja", [[-79.918242,-2.170096], [-79.915066,-2.167523]]],
             ["Samborondon", [[-79.874897,-2.160447], [-79.852066,-2.116704]]],
             ["Aeropuerto", [[-79.886227,-2.152728], [-79.880562,-2.146338]]],
             ["Penias", [[-79.8776014239,-2.1855487464], [-79.873009,-2.175392]]],
             ["Quito", [[-78.586922,-0.395161], [-78.274155,0.021973]]],
             ["CentroConvenciones", [[-79.893479,-2.161755], [-79.883652,-2.154894]]],
             ["HiltonColon", [[-79.8998951912,-2.160929558], [-79.8962903023,-2.1576274229]]],
             ["MallSol", [[-79.8954641819,-2.157053837], [-79.8891341686,-2.1539258197]]],
             ["UniversidadGuayaquil", [[-79.9008607864,-2.1847947751], [-79.8959684372,-2.1791554991]]]
            ]

In [8]:
def localizar(row, places):
    longitud = float(row["longitud"])
    latitud = float(row["latitud"])

    for placeName, ubicacion in places:
        SW, NE= ubicacion
        if (latitud <= NE[0] and longitud <=NE[1] and latitud >= SW[0] and longitud >= SW[1]):
            return placeName
    return "No ubicado"

In [9]:
sitios['localizacion'] = sitios[["longitud", "latitud"]].apply(lambda row: localizar(row, places), axis = 1)
sitios.head()

,sitio_id,nombre,siglas,ruc,ciudad,nombre_ubicacion,contacto,direccion,email,telefono,...,longitud,latitud,ip,puerto,fecha,usuario_modif,bienvenida,formulario,router,localizacion
0,1,ALBAN BORJA,ALBAN BORJ,920124541001,Guayaquil,Guayaquil,Fabricio Matamoros,Av. Carlos Julio Arosemena,administracion@albanborja.com,923154211,...,-2.168677,-79.916769,186.3.137.95,NaN,2016-09-19 13:28:32,fmatamoros,0,0,Mikrotik,AlbanBorja
1,2,VILLAGE PLAZA,VILLA,0,Guayaquil,Guayaquil,Gabriela,SAMBORONDON,NaN,NaN,...,-2.140406,-79.866376,186.3.144.18,NaN,2016-07-20 17:23:50,fmatamoros,0,0,NaN,Samborondon
2,3,CITY MALL,CITY M.,0,Guayaquil,Guayaquil,Gabriela,Alborada,NaN,NaN,...,-2.140951,-79.911774,186.3.153.140,NaN,2015-12-22 10:21:54,fmatamoros,0,0,NaN,CityMall
3,4,MALL DEL SUR,MALL DEL S,0,Guayaquil,NaN,Gabriela,Sur,mtorres@seruvi.com,04 3068130,...,-2.229176,-79.900660,181.39.198.134,NaN,2015-12-23 16:03:23,charvel.chedraui,0,0,NaN,MallSur
4,5,XS DISCO,XS,0,Guayaquil,Guayaquil,Fabricio Matamoros,Victor E. Estrada 819 e Higueras,nd,nd,...,-2.166894,-79.913704,186.3.137.86,NaN,2015-12-16 16:59:55,charvel.chedraui,0,0,NaN,Urdesa


In [10]:
sitios2 = sitios[["sitio_id", "localizacion"]] 
sitios2.head(3)

,sitio_id,localizacion
0,1,AlbanBorja
1,2,Samborondon
2,3,CityMall


# UNIENDO LOS DATAFRAMES 

In [11]:
datos = pd.merge(conexiones2, sitios2, on = "sitio_id")
datos.head()

,datetime,mac_usr,sitio_id,localizacion
0,2015-05-15 20:15:00,94:01:C2:97:96:CB,2,Samborondon
1,2015-05-15 20:15:00,8C:BF:A6:17:60:A0,2,Samborondon
2,2015-05-15 20:15:00,34:BE:00:05:16:18,2,Samborondon
3,2015-05-15 20:15:00,BC:6E:64:71:73:97,2,Samborondon
4,2015-05-15 20:15:00,B8:6C:E8:C3:F8:C9,2,Samborondon


In [12]:
datos2 = datos.groupby(["localizacion", "datetime"])
archivo = open("todo.csv", "w")
archivo.write("Mac1,Mac2\n")
lugares = {"todo": archivo}
for nombre, dato in datos2:
    if nombre[0] not in lugares:
        lugares[nombre[0]] =open(nombre[0].lower() +".csv", "w")
        lugares[nombre[0]].write("Mac1,Mac2\n")
    aux = lugares[nombre[0]]
    macs = list(set(list(dato["mac_usr"])))
    for i in range(len(macs)):
        for j in range(i+1, len(macs)):
            lista = [macs[i], macs[j]]
            lista.sort()
            mac1,mac2 = lista
            aux.write(mac1 + "," + mac2 + "\n")
            archivo.write(mac1 + "," + mac2 + "\n")
            
for k,v in lugares.items():
    v.close()
"Termino"


'Termino'

In [19]:
for lugar in lugares:
    arcos = pd.DataFrame.from_csv(lugar.lower() +".csv", sep = ",").reset_index()
    aux = arcos.groupby(["Mac1", "Mac2"]).size().rename("peso").reset_index()
    aux = aux.sort_values("peso", ascending = False)
    aux.to_csv(lugar.lower() +"Peso.csv", sep=',', index = False)
    print(lugar)

todo
Aeropuerto
AlbanBorja
CentroConvenciones
CityMall
HiltonColon
MallSol
MallSur
Penias
Quito
Samborondon
UniversidadGuayaquil
Urdesa
